In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'
import re
import sys
from operator import add
from typing import Iterable,Tuple
from pyspark.resultiterable import ResultIterable
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv("gs://csee4121homework/outputs/p1t2_small.csv",sep = "\t").dropna(subset=["_c1"])

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d0d26296-cb0c-4d28-9438-4e6e357e205c;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 283ms :: artifacts dl 8ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
def parse(urls):
    return (urls[0],urls[1])

def computecontribution(neighbors,rank):
    for neighbor in neighbors:
        yield (neighbor, rank/len(neighbors))


links = df.rdd.map(lambda line:parse(line))
links = links.distinct().groupByKey()

##initialize rank to 1
ranks = links.map(lambda url_neighbors:(url_neighbors[0],1.0))

for i in range(10):
    contributions = links.join(ranks).\
    flatMap(lambda rank: computecontribution(rank[1][0],rank[1][1])) #individual contributions
    ### map null values to 0
    contributions = links.fullOuterJoin(contributions).\
    mapValues(lambda neighbors: neighbors[1] or 0)
    
    ranks = contributions.reduceByKey(add).mapValues(lambda rank:rank*0.85 + 0.15)




In [4]:
ranks.toDF(["_c0","_c1"]).withColumnRenamed("_c0","articles").withColumnRenamed("_c1","links")\
.dropna().repartition(1).orderBy(["articles","links"]).limit(5).\
write.csv("gs://csee4121homework/outputs/p1t3.csv",mode="overwrite",sep = "\t")